# Importation des librairies

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

In [ ]:
# Chargement de l'ensemble de données
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0272EN-SkillsNetwork/labs/dataset/2016.csv"
df = pd.read_csv(URL)
# Lire les cinq premières lignes de l'ensemble de données
df.head()

# Nettoyage et exploration

In [ ]:
# Dimession de l'ensemble de données
df.shape

In [ ]:
# Vérification des types de données
types_attendus = df.dtypes.to_dict()

In [ ]:
types_attendus

In [ ]:
# Nettoyage du DataFrame
for col in df.columns:
    # Supprimer les espaces de début et de fin pour les colonnes de type objet (chaînes de caractères)
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip()

        # Remplacer les chaînes vides par des valeurs NaN
        df[col] = df[col].replace('', np.nan)

    # Tenter de convertir les colonnes en numérique (si possible)
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        pass

print("\nDataFrame après nettoyage :")
df.head(10)



In [ ]:
print("\nTypes de données après conversion :")
df.dtypes.to_dict()

In [ ]:
# Identifier et traiter les colonnes avec des valeurs manquantes
for col in df.columns:
    if df[col].isnull().any():  # Vérifie si la colonne contient des NaN
        if df[col].dtype == 'object' or df[col].dtype.name == 'category':
            # Remplacer les valeurs manquantes par le mode pour les colonnes catégorielles
            mode = df[col].mode()[0]  # Le mode retourne une série, on récupère le premier élément
            df[col].fillna(mode, inplace=True)
        else:
            # Remplacer les valeurs manquantes par la moyenne pour les colonnes numériques
            mean = df[col].mean()
            df[col].fillna(mean, inplace=True)

In [ ]:
# Les valeurs manquantes sont bien traitées
sns.heatmap(df.isnull(), cbar=False)
plt.show()

In [10]:
# Créer un graphique à barres avec Plotly
fig1 = go.Figure()

# Ajouter les barres pour le PIB par habitant
fig1.add_trace(go.Bar(
    x=df['Country'],
    y=df['Economy (GDP per Capita)'],
    name='PIB par habitant',
    marker_color='black'
))

# Ajouter les barres pour l'espérance de vie en bonne santé
fig1.add_trace(go.Bar(
    x=df['Country'],
    y=df['Health (Life Expectancy)'],
    name='Espérance de vie en bonne santé',
    marker_color='red'
))

# Mettre à jour la mise en page du graphique
fig1.update_layout(
    title='PIB par habitant et Espérance de vie en bonne santé des 10 premiers pays',
    xaxis_title='Pays',
    yaxis_title='Valeurs',
    barmode='group',
    template='plotly_white'
)

# Afficher le graphique
fig1.show()

In [11]:
df.columns

Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'],
      dtype='object')

In [12]:
# 1. Créer un sous-ensemble de données
attributs = [
    'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
    'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Happiness Score'
]
subset = df[attributs]

# 2. Calculer la corrélation
correlation_matrix = subset.corr()

# Créer une carte thermique de corrélation avec Plotly
fig2 = go.Figure()

# Ajouter les valeurs de la matrice de corrélation comme graphique à chaud
fig2.add_trace(go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    colorscale='Viridis',
    colorbar=dict(title='Corrélation')
))

# Mettre à jour la mise en page
fig2.update_layout(
    title="Carte thermique des corrélations",
    width=800,
    height=600,
    xaxis_title="Attributs",
    yaxis_title="Attributs",
    template="plotly_white"
)

# Afficher la carte thermique
fig2.show()

In [17]:

# Créer le nuage de points avec Plotly Express
fig3 = px.scatter(
    df,
    x='Economy (GDP per Capita)',
    y='Happiness Score',
    color='Region',
    hover_name='Country',  # Affiche le nom du pays au survol
    title="Relation entre le PIB par habitant et le Score de bonheur dans différentes régions",
    labels={'GDP per Capita': 'PIB par habitant ($)', 'Happiness Score': 'Score de bonheur'}
)

# Mettre à jour la mise en page
fig3.update_layout(
    width=800,  # Largeur de la figure
    height=600,  # Hauteur de la figure
    template='plotly_white'
)

# Afficher le graphique
fig3.show()

In [19]:
# Créer le diagramme circulaire avec Plotly Express
fig4 = px.pie(
    df,
    names='Region',  # Colonne avec les catégories (Région)
    values='Happiness Score',  # Colonne avec les valeurs numériques
    title='Score de Bonheur par Région',
    color_discrete_sequence=px.colors.qualitative.Pastel  # Optionnel : Définit le schéma de couleurs
)

# Mettre à jour la mise en page pour une meilleure visualisation
fig4.update_layout(
    width=800,  # Largeur de la figure
    height=600,  # Hauteur de la figure
    template='plotly_white'  # Optionnel : Applique un modèle clair
)

# Afficher le graphique
fig4.show()

In [20]:
df.columns

Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'],
      dtype='object')

In [23]:
# Créer la carte avec Plotly Express
fig5 = px.choropleth(
    df,
    locations='Country',  # Utilise les noms des pays pour la géolocalisation
    locationmode='country names',  # Spécifie que les noms des pays sont utilisés
    color='Economy (GDP per Capita)',  # Valeur pour les couleurs (PIB par habitant)
    hover_name='Country',  # Affiche le nom des pays
    hover_data={'Health (Life Expectancy)': True},  # Inclut l'espérance de vie dans les infobulles
    title="Carte interactive : PIB par habitant et Espérance de vie en bonne santé",
    labels={'PIB par habitant': 'PIB par habitant ($)', 'Espérance de vie en bonne santé': 'Espérance de vie'}
)

# Ajuster la mise en page de la carte
fig5.update_layout(
    width=800,  # Largeur de la figure
    height=600,  # Hauteur de la figure
    geo=dict(showframe=False, showcoastlines=True, projection_type='equirectangular'),
    template="plotly_white"
)

# Afficher la carte
fig5.show()

In [31]:
# Create a single HTML file containing all figures
figures = [fig1, fig2, fig3, fig4, fig5]
with open('dashboard.htm', 'w') as f:
    for fig in figures:
        f.write(pio.to_html(fig, full_html=False))


Présentation du Tableau de Bord du Rapport sur le Bonheur Mondial

Bienvenue dans le tableau de bord interactif du Rapport sur le Bonheur Mondial. Cet outil visuel offre une exploration approfondie des facteurs influençant le bonheur mondial, en fournissant des perspectives claires et engageantes grâce à quatre visualisations distinctes :

Carte Thermique des Corrélations Plongez dans l’analyse des relations entre plusieurs indicateurs clés, tels que le PIB par habitant, l'espérance de vie en bonne santé, la confiance dans les institutions et la générosité. La carte thermique met en évidence les corrélations positives et négatives entre ces variables, permettant d'identifier les interactions les plus significatives dans l'ensemble des données.

Nuage de Points : PIB par Habitant vs. Score de Bonheur Découvrez comment le PIB par habitant impacte le score de bonheur à travers différentes régions. Ce nuage de points interactif permet de visualiser les disparités régionales et met en lumière les pays où un PIB élevé coïncide avec un score de bonheur important. Les couleurs représentent les régions, rendant cette analyse encore plus intuitive.

Diagramme Circulaire : Score de Bonheur par Région Cette visualisation présente une répartition claire du score de bonheur entre les différentes régions du monde. Elle met en avant les contributions relatives des régions au bonheur mondial, offrant une perspective globale sur les différences géographiques.

Carte Interactive : PIB par Habitant et Espérance de Vie en Bonne Santé Explorez une carte mondiale affichant le PIB par habitant des pays avec une échelle de couleurs intuitive. En survolant chaque pays, les utilisateurs peuvent accéder à des infobulles contenant des informations supplémentaires, telles que l'espérance de vie en bonne santé, fournissant ainsi un contexte enrichi à chaque donnée.

Ce tableau de bord est conçu pour faciliter l’analyse et approfondir la compréhension des facteurs qui influencent le bonheur mondial. Il constitue un outil puissant pour les chercheurs, les décideurs et tous ceux qui s'intéressent au bien-être global. Prenez le temps d'explorer ces graphiques interactifs pour découvrir les tendances et les relations qui façonnent le bonheur à travers le *monde*